<a href="https://colab.research.google.com/github/TheSanjBot/Apparel-Classifier/blob/main/CNN_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
train_images


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
    model = keras.Sequential([
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=(3, 3),
            activation="relu"
        ),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
            kernel_size=(3, 3),
            activation="relu"
        ),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)),
        layers.Dense(10, activation="softmax"),
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
!pip install -U keras-tuner

In [ ]:
from keras_tuner import RandomSearch
from keras_tuner import HyperParameters
from tensorflow.keras import layers

In [ ]:
tuner_search = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='output',
    project_name='mnist_tuning'
)

In [ ]:
tuner_search.search(train_images,train_labels,validation_split=0.1,epochs=3)

Trial 5 Complete [00h 08m 12s]
val_accuracy: 0.856166660785675

Best val_accuracy So Far: 0.8956666588783264
Total elapsed time: 00h 28m 28s


In [ ]:
tuner_search.search(train_images,train_labels,validation_split=0.1,epochs=3)

In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 80)        800       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 80)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 112)       80752     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 112)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2800)              0         
                                                                 
 dropout (Dropout)           (None, 2800)              0

In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 128s 75ms/step - loss: 0.2657 - accuracy: 0.9038 - val_loss: 0.2706 - val_accuracy: 0.9038
Epoch 5/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.2438 - accuracy: 0.9109 - val_loss: 0.2656 - val_accuracy: 0.9002
Epoch 6/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.2289 - accuracy: 0.9156 - val_loss: 0.2643 - val_accuracy: 0.9025
Epoch 7/10
1688/1688 [==============================] - 125s 74ms/step - loss: 0.2125 - accuracy: 0.9224 - val_loss: 0.2441 - val_accuracy: 0.9128
Epoch 8/10
1688/1688 [==============================] - 125s 74ms/step - loss: 0.2021 - accuracy: 0.9264 - val_loss: 0.2456 - val_accuracy: 0.9150
Epoch 9/10
1688/1688 [==============================] - 126s 75ms/step - loss: 0.1887 - accuracy: 0.9305 - val_loss: 0.2481 - val_accuracy: 0.9128
Epoch 10/10
1688/1688 [==============================] - 126s 74ms/step - loss: 0.1803 - accuracy: 0.9334 - val_loss: 